In [3]:
# !gdown 1jwJqh3CA6Grcykoh-AA8LlUfyamdFuty

Downloading...
From: https://drive.google.com/uc?id=1jwJqh3CA6Grcykoh-AA8LlUfyamdFuty
To: /content/model.joblib
100% 1.70M/1.70M [00:00<00:00, 140MB/s]


In [2]:
# !pip install sklearn_crfsuite
import joblib
import pandas as pd
model = joblib.load("model.joblib")

In [3]:
stopwords = ["ผู้", "ที่", "ซึ่ง", "อัน"]

def tokens_to_features(tokens, i):
  word = tokens[i]

  features = {
    "bias": 1.0,
    "word.word": word,
    "word[:3]": word[:3],
    "word.isspace()": word.isspace(),
    "word.is_stopword()": word in stopwords,
    "word.isdigit()": word.isdigit(),
    "word.islen5": word.isdigit() and len(word) == 5
  }

  if i > 0:
    prevword = tokens[i - 1]
    features.update({
      "-1.word.prevword": prevword,
      "-1.word.isspace()": prevword.isspace(),
      "-1.word.is_stopword()": prevword in stopwords,
      "-1.word.isdigit()": prevword.isdigit(),
    })
  else:
    features["BOS"] = True

  if i < len(tokens) - 1:
    nextword = tokens[i + 1]
    features.update({
      "+1.word.nextword": nextword,
      "+1.word.isspace()": nextword.isspace(),
      "+1.word.is_stopword()": nextword in stopwords,
      "+1.word.isdigit()": nextword.isdigit(),
    })
  else:
    features["EOS"] = True

  return features

def parse(text):
  tokens = text.split()
  features = [tokens_to_features(tokens, i) for i in range(len(tokens))]
  return model.predict([features])[0]

In [4]:
parse("นายสมชาย เข็มกลัด 254 ถนน พญาไท แขวง วังใหม่ เขต ปทุมวัน กรุงเทพมหานคร 10330")

array(['O', 'O', 'ADDR', 'ADDR', 'ADDR', 'ADDR', 'ADDR', 'ADDR', 'LOC',
       'LOC', 'POST'], dtype=object)

In [5]:
parse("นายมงคล 123/4 ตำบล บ้านไกล อำเภอ เมือง จังหวัด ลพบุรี 15000")

array(['ADDR', 'ADDR', 'ADDR', 'ADDR', 'ADDR', 'ADDR', 'ADDR', 'ADDR',
       'POST'], dtype=object)

In [6]:
with open('NE-dict.txt', 'r') as f:
    lines = f.readlines()

# Remove newline characters from each line
lines = [line.strip() for line in lines]

FileNotFoundError: [Errno 2] No such file or directory: 'NE-dict.txt'

In [7]:
for i in range(200):
  tag = parse(lines[i])
  print(f'{tag}, {lines[i]}')

NameError: name 'lines' is not defined

In [8]:
with open('NE-dict.txt', 'r') as f:
    lines = f.readlines()

# Remove newline characters from each line
lines = [line.strip() for line in lines]

# Create a new list to store lines containing 'เขต'
lines_district = []

# Iterate through each line and add to the new list if it contains 'เขต'
for line in lines:
    if 'เขต' in line:
        lines_district.append(line)

FileNotFoundError: [Errno 2] No such file or directory: 'NE-dict.txt'

In [9]:
for i in range(len(lines_district)):
  tag = parse(lines_district[i])
  print(f'{tag}, {lines_district[i]}')

NameError: name 'lines_district' is not defined

In [10]:
parse('เทศบาลตำบลตลาดเขต อำเภอพนมทวน จังหวัดกาญจนบุรี')

array(['ADDR', 'ADDR', 'ADDR'], dtype=object)

In [11]:
import zipfile
from bs4 import BeautifulSoup
import textwrap
import pandas as pd

zipf = zipfile.ZipFile('khaosod.zip','r')
filenames = zipf.namelist()

corpus_content = list()
for filename in filenames:
  html_file = zipf.open(filename)
  html_string = html_file.read().decode('utf-8')
  html_file.close()

  html_soup = BeautifulSoup(html_string, 'html.parser')
  # print(html_soup.prettify())

  title_box = html_soup.find('h1',class_='udsg__main-title')
  title = title_box.text.strip()
  # print(title)

  content_box = html_soup.find('div', class_='udsg__content')
  content = ''
  for p in content_box.find_all('p'):
    content += p.text
  content = content.replace('\n','').replace('  ','').strip()
  # print(textwrap.fill(content,width=120))

  corpus_content.append({'title':title,
                         'content':content
                         })
zipf.close()

df_corpus = pd.DataFrame(corpus_content)
df_corpus.info()
display(df_corpus.head())

df_corpus.to_json('khaosod.json', orient = 'table', force_ascii = False, indent = 2)
print('done')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    52 non-null     object
 1   content  52 non-null     object
dtypes: object(2)
memory usage: 964.0+ bytes


,title,content
0,หลาก&หลายไอที - อลังการไลน์อัพโน้ตบุ๊กAI เลอโ...,เลอโนโวเปิดตัวผลิตภัณฑ์ใหม่ล่าสุดของไลน์อัพ Le...
1,หลาก&หลายไอที - อลังการแอปเปิ้ลอีเวนต์ ชูไอโฟ...,นอกจากสินค้าเรือธงอย่าง iPhone 16 series ซึ่งแ...
2,สาวกห้ามพลาด เทียบราคา iPhone16 ถูกกว่า iPhone15,สาวก Apple ห้ามพลาด เทียบราคา iPhone16 ถูกกว่า...
3,หลาก&หลายไอที - ‘แผ่นพิมพ์วงจรไฟฟ้า’โอกาสไทย-ผ...,ประเทศไทยกำลังอยู่ในช่วงหัวเลี้ยวหัวต่อสำคัญขอ...
4,หลาก&หลายไอที - ‘แผ่นพิมพ์วงจรไฟฟ้า’ โอกาสไทย...,แผ่นพิมพ์วงจรไฟฟ้าหรือ พีซีบีเป็นรากฐานที่สำคั...


done


In [12]:
df_corpus['content']

0     เลอโนโวเปิดตัวผลิตภัณฑ์ใหม่ล่าสุดของไลน์อัพ Le...
1     นอกจากสินค้าเรือธงอย่าง iPhone 16 series ซึ่งแ...
2     สาวก Apple ห้ามพลาด เทียบราคา iPhone16 ถูกกว่า...
3     ประเทศไทยกำลังอยู่ในช่วงหัวเลี้ยวหัวต่อสำคัญขอ...
4     แผ่นพิมพ์วงจรไฟฟ้าหรือ พีซีบีเป็นรากฐานที่สำคั...
5     เลอโนโว ผู้พัฒนาผลิตภัณฑ์คอมพิวเตอร์ชื่อดังจาก...
6     ซัมซุงผู้พัฒนาสมาร์ตโฟนตระกูลกาแล็กซีชื่อก้องโ...
7     เอซุส (ประเทศไทย)ร่วมกับ AMDเปิดตัวโน้ตบุ๊กและ...
8     เดลล์ผู้พัฒนาผลิตภัณฑ์คอมพิวเตอร์ชื่อดังจากประ...
9     ชาวเน็ตทั่วโลกฮือฮา ! Sonny Dickson ปล่อยภาพ i...
10    ตลาดโน้ตบุ๊กเป็นหนึ่งในตลาดที่มีการแข่งขันสูง ...
11    เอซุส ผู้พัฒนาสินค้าไอทีชื่อก้องโลกจากไต้หวันท...
12    ไมโครซอฟท์– วันที่ 4 ก.ค. รอยเตอร์รายงานว่า ไม...
13    แอปเปิ้ลหนึ่งในผู้พัฒนาสินค้าไอทีชั้นนำของโลกจ...
14    เลอโนโว พร้อมวางจำหน่ายโน้ตบุ๊ก Lenovo Yoga Bo...
15    วิศวกรไฟฟ้าวัย 61 ปี ไม่น่าจะมีแฟนๆ รุมกรี๊ด ล...
16    แอปเปิ้ลประกาศเปิดตัว iPad Air รุ่น 11 และ 13 ...
17    Apple เผยโฉม iPad Pro ใหม่สุดล้ำ ดีไซน์เบา

In [52]:
cont = []
result = []
for i in range(df_corpus.shape[0]):
    content = df_corpus['content'][i]
    cont.append(content.split())
    result.append(parse(content).tolist())

cont = sum(cont,[])
result = sum(result,[])

In [53]:
result_table = pd.DataFrame({'token': cont,
                             'tag': result})
display(result_table)

,token,tag
0,เลอโนโวเปิดตัวผลิตภัณฑ์ใหม่ล่าสุดของไลน์อัพ,ADDR
1,Lenovo,ADDR
2,Yogaและ,ADDR
3,Lenovo,ADDR
4,IdeaPadผ่านการออกแบบเพื่อเพิ่มประสิทธิภาพการทำ...,ADDR
...,...,...
14603,การคัดกรองประวัติ,ADDR
14604,การสอบข้อเขียน,ADDR
14605,การสัมภาษณ์,ADDR
14606,ฯลฯที่มา:,ADDR


In [54]:
tag_counts = result_table['tag'].value_counts().reset_index()